# CHATBOT FOR A RADIOLOGY CENTER APPOINTMENTS BOOKING 
THIS NOTEBOOK CONTAINS STEP BY STEP INSTRUCTIONS TO DEVELOP A CHATBOT FOR A SPECIFIC RADIOLOGY CENTER, THE CHATBOT IS ABLE
TO COMMUNICATE AND OFFER  DOCTORS, TESTS AND APPOINTMENTS AVAILABLE, ALL THE APPOINTMENTS BOOKED OR DENIED ARE STORED IN AN EXCEL FILE.
THE CHATBOT IS DEVELOPED USING GPT 3.5 API OF OPENAI, AND THE OBJECTIVE OF THIS PROJECT IS TO DEMONSTRATE THE ADVANTAGES OF USING NEW AI TECNOLOGIES IN SPECIALIZED CHATBOTS.

FOR THIS PROJECT WE WILL USE A RADIOLOGY CENTER WITH FOUR DOCTORS, THE CENTER ALSO OFFER THREE DIFFERENT RADIOLOGY TESTS. DOCTORS AND TESTS CAN BE BOOKED BY OUR CHATBOT, AND HE CAN DENY AN APPOINTMENT IN CASE OF NON-AVAILABILITY

SAENZ RADIOLOGY CENTER:

Dr. Emily Carter, Specialization: Diagnostic Radiology, Cost 500 
Dr. Pedro Saenz, Specialization: Interventional Radiology, Cost 1000 
Dr. Sarah Gonzalez, Specialization: Musculoskeletal Radiology, Cost 300 
Dr. Aisha Rahman, Specialization: Neuroradiology, Cost 350 
Radiology Tests
MRI Test, Cost 2000 
CT Scanner Test, Cost 3000 
X-Ray Test, Cost 500
The working hours are from 2pm to 9pm.

Created by Jorge Saenz

You will first import some libraries, openai is the tool we will use in this project.
the openai.api_key is personal, and it can be found in openai-account-keys.

In [1]:
import openai
import pandas as pd
import regex as re

openai.api_key  ='sk-NnKuSP5Wb8N4tVXwfFx6T3BlbkFJG4tNyASGlC7phwxxxxx'

Now let's create two functions to acces gpt 3.5, this functions are provide by openai.
Both functions recived a prompt and the ID of the model to use (models can be found in openai), in function get_completion, 
we create a conversation where we name the role and the content of the message. This way are indicating the AI its role 
and the content of the action.

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo-16k-0613"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

Now we will import the files xlsx with the database of the appointments, we will store this data in a pd dataframe.

In [3]:
data_doctors=pd.read_excel("Doctors_appointments.xlsx", header=0)
data_test=pd.read_excel("Test_appointments.xlsx", header=0)

In [4]:
read_data=pd.read_excel("Doctors_appointments.xlsx", header=0)
read_data

,Booked,Patient,Day,Hour,Doctor
0,False,name,week day,hour,Doctor
1,True,Jorge,Friday,4pm,Dr. Emily Carter
2,True,Lopez,Wednesday,6pm,Dr. Aisha Rahman
3,True,Andrea,Thursday,7pm,Dr. Pedro Saenz
4,False,Lupita,Wednesday,6pm,Dr. Aisha Rahman
5,True,Gabriela,Friday,7pm,Dr. Pedro Saenz
6,True,Jorge,Tuesday,5pm,Dr. Pedro Saenz
7,False,Josh,Wednesday,6pm,Dr. Aisha Rahman
8,True,Marshall,Friday,3pm,Dr. Emily Carter
9,True,Miguel,jueves,7pm,Dr. Pedro Saenz


Also, if you want to create a new database  or to clean the previous, you can run the next cell that creates a clean dataset.

In [ ]:
columns_doctors = ['Booked','Patient', 'Day', 'Hour', 'Doctor']
data_doctors = pd.DataFrame(columns=columns_doctors)

initiate=[False,'name','week day','hour','Doctor']
data_doctors=data_doctors.append(pd.Series(initiate, index=columns_doctors), ignore_index=True)
columns_test = ['Booked','Patient', 'Day', 'Hour', 'Test']
data_test = pd.DataFrame(columns=columns_test)
initiate=[False,'name','week day','hour','Test']
data_test=data_test.append(pd.Series(initiate, index=columns_test), ignore_index=True)
data_doctors

Now lets create a function to save the new appointments in our pandas dataframe.
The function will do the following:

    -Find a patter for the confirmation code of the appointment given by 
    the chatbot (name of patient/date/time/Doctor or test).
    -We will call this function over each response of the chatbot, so it will only process 
    the text until it finds the pattern.
    -Now we will process the confirmation code and extract each item, ask if it is for a Doctor or a Test, split it and
    save it into the dataframe.
    -We will check if the appointment is available and return ¨Available¨ or ¨Busy¨.
    

In [ ]:
def save_appointment(response):
    global data_doctors
    global data_test 
    availability='Available'
    pattern =r'\((.*?)\)'
    code = re.findall(pattern, response)
    if len(code)==1:
        data=code[0].split("/")
        data.insert(0,False)
        if ((data[4].endswith("Test")) or (data[4].endswith("test")) or (data[4].endswith("Scanner")) or (data[4].endswith("MRI")) or (data[4].endswith("X-Ray"))):
            if data[4]=="MRI": data[4]="MRI Test"
            if data[4]=="MRI test": data[4]="MRI Test"
            if data[4]=="X-Ray": data[4]="X-Ray Test"
            if data[4]=="X-Ray test": data[4]="X-Ray Test"
            if data[4]=="CT Scanner": data[4]="CT Scanner Test"
            if data[4]=="CT Scanner test": data[4]="CT Scanner Test"
            if not data_test[(data_test['Day'] == data[2]) & (data_test['Hour'] == data[3] ) & (data_test['Test'] == data[4] )].empty:
                availability='Busy'
                data[0]=False
                data_test= data_test.append(pd.Series(data, index=columns_test), ignore_index=True)
            else:
                availability='Available'
                data[0]=True
                data_test= data_test.append(pd.Series(data, index=columns_test), ignore_index=True)
        else:
            if data[4]=="Diagnostic Radiology": data[4]="Dr. Emily Carter"
            if data[4]=="Interventional Radiology": data[4]="Dr. Pedro Saenz"
            if data[4]=="Musculoskeletal Radiology": data[4]="Dr. Sarah Gonzalez"
            if data[4]=="Neuroradiology": data[4]="Dr. Aisha Rahman"
            if not data_doctors[(data_doctors['Day'] == data[2]) & (data_doctors['Hour'] == data[3] ) & (data_doctors['Doctor'] == data[4] )].empty:
                availability='Busy'
                data[0]=False
                data_doctors = data_doctors.append(pd.Series(data, index=columns_doctors), ignore_index=True)
            else:    
                availability='Available'
                data[0]=True
                data_doctors = data_doctors.append(pd.Series(data, index=columns_doctors), ignore_index=True)
    return availability

We will make use of the library Panel, to display our chatbot.
We need to create a function for each time we click a button (when the user responds), this function will do the following:

    -Extract the input value from the input space
    -We need to create the context, to include each response and interaction between the user and the chatbot.
    -This context will be send to the function save_appointment, this will give all the details and the context of the
    conversation to the AI.
    -We need to call the function save_appointment for the AI reponse, in order to detect a booked confirmation.
    -Finally, we need to use Panel to show the interaction between user and chatbot.

In [5]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    availability=save_appointment(response)
    if (availability=='Available'):
        context.append({'role':'assistant', 'content':f"{response}"})
    elif(availability=='Busy'):
        response="My apologies, your appointment could not be booked, try a different day/date "
        context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Chatbot:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 
context = [ {'role':'system', 'content':"""
You are ChatBot, an automated service to assist patients \
in booking appointments in the Saenz Radiology center for both specialized \
doctors and various radiology tests. \
You first greet the customer and welcome him to the center, ask if they need an appointment for a \
Doctor or a radiology test,they can only choose a Doctor or a test, then you show the Doctors or tests available, \
then you ask for the day of the appointmen, then you ask for the hour, and 
then the nameof the patient.\
You respond in a short, very conversational friendly style.
Make sure to clarify all Doctors available or tests.\
You wait to collect all the info, then you will check if the info was correct, if the patient \
confirms the appointment, you will only responde with comfirmation code inside parentheses,\
the comfirmation code format is: (name of patient/date/time/Doctor or test) \
separated by a slash. \
The Doctors are \
Dr. Emily Carter, Specialization: Diagnostic Radiology, Cost 500 \
Dr. Pedro Saenz, Specialization: Interventional Radiology, Cost 1000 \
Dr. Sarah Gonzalez, Specialization: Musculoskeletal Radiology, Cost 300 \
Dr. Aisha Rahman, Specialization: Neuroradiology, Cost 350 \
Radiology Tests\
MRI Test, Cost 2000 \
CT Scanner Test, Cost 3000 \
X-Ray Test, Cost 500
The working hours are from 2pm to 9pm.
"""} ]  # accumulate messages

title_html = """
<link href="https://fonts.googleapis.com/css2?family=Montserrat:wght@500&display=swap" rel="stylesheet">
<div style="text-align: center; background-color: #0074E4; padding: 20px;">
    <div style="display: flex; align-items: center; justify-content: center;">
        <img src="radiology_logo.jpg" alt="Radiology Symbol" style="height: 60px; margin-right: 10px;">
        <h1 style="font-family: 'Montserrat', sans-serif; color: white; font-size: 30px;">Saenz Radiology Center</h1>
    </div>
</div>
"""
#title = pn.pane.Markdown("# Saenz Radiology Center ", styles={'font-size': '24px', 'font-weight': 'bold'})
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    pn.panel(title_html, sizing_mode='stretch_width'),
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Now, this is optional, and it must be included in save_appointment function after you complete your debbuging, 
the next to cells show the code to save the dataframe used in the notebook into a excel file. It will rewrite the 
existen xlsx file with the new appointments.

In [ ]:
file_path = 'Test_appointments.xlsx'

# Save the DataFrame to an Excel file
data_test.to_excel(file_path, index=False)

In [ ]:
file_path = 'Doctors_appointments.xlsx'

# Save the DataFrame to an Excel file
data_doctors.to_excel(file_path, index=False)